# Variables

In [86]:
OPENAI_API_KEY='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'
FIRECRAWL_API_KEY='fc-e36282d62d1c4ee8971ec0649f3862c7'
BASE_URL='https://api.groq.com/openai/v1'
MODEL="meta-llama/llama-4-scout-17b-16e-instruct"

# Import Modules

In [87]:
import sys
sys.path.append('..')
print(sys.path)
from tools.fetching_description_from_huggingface import fetching_description_from_huggingface
from typing import TypedDict, Annotated, List
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.agents import create_tool_calling_agent, AgentExecutor
from pydantic import BaseModel, Field
from langchain.tools import Tool
from firecrawl import FirecrawlApp
from langchain_core.output_parsers import PydanticOutputParser
import pandas as pd
import pprint

['c:\\Users\\viett\\AppData\\Local\\Programs\\Python\\Python313\\python313.zip', 'c:\\Users\\viett\\AppData\\Local\\Programs\\Python\\Python313\\DLLs', 'c:\\Users\\viett\\AppData\\Local\\Programs\\Python\\Python313\\Lib', 'c:\\Users\\viett\\AppData\\Local\\Programs\\Python\\Python313', '', 'c:\\Users\\viett\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages', 'C:\\Users\\viett\\label-studio', 'c:\\Users\\viett\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\win32', 'c:\\Users\\viett\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\viett\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\Pythonwin', '..', '..', '..', '..', '..', '..']


# Testing

In [88]:
df_task = pd.read_csv("data/ISE - AutoCode Challenge 2 Public - Public task.csv")
df_model = pd.read_csv("data/ISE - AutoCode Challenge 2 Public - Model zoo.csv")

In [89]:
df_task

,STT,Task,Validation data
0,1,"""Bối cảnh của vấn đề:\r\nNhận dạng chữ viết ta...",Link
1,2,Bối cảnh của vấn đề:\r\nTrong thời đại thông t...,Link
2,3,Bối cảnh của vấn đề:\r\nĐây là một nhiệm vụ hồ...,Link
3,4,Bối cảnh của vấn đề:\r\nĐiều quan trọng là các...,Link
4,5,Bối cảnh của vấn đề:\r\nTrong nhiều ứng dụng t...,Link


In [90]:
task = df_task["Task"][0]

In [91]:
task

'"Bối cảnh của vấn đề:\r\nNhận dạng chữ viết tay là một bài toán cơ bản trong lĩnh vực học máy và xử lý ảnh, với nhiều ứng dụng thực tiễn như nhận dạng chữ số trên phiếu khảo sát, hóa đơn hay bài thi tự động. Việc phân loại chữ số viết tay thành số nguyên tố hoặc không giúp mở rộng khả năng ứng dụng trong các bài toán toán học tự động, kiểm tra bài tập, hoặc các ứng dụng giáo dục.\r\n\r\nYêu cầu cụ thể cần đạt được:\r\nXây dựng một hệ thống có khả năng nhận diện chữ số viết tay từ ảnh và xác định xem số đó có phải là số nguyên tố hay không.\r\nĐầu vào là ảnh chứa một chữ số viết tay.\r\nĐầu ra là nhãn phân loại nhị phân: ""nguyên tố"" hoặc ""không nguyên tố"".\r\n\r\nĐịnh dạng dữ liệu đầu vào cho bài toán tổng thể:\r\nMột thư mục có tên là ""images"" chứa các ảnh grayscale 28x28 pixel, mỗi ảnh chứa một chữ số viết tay từ 0 đến 9.\r\n\r\nĐịnh dạng kết quả đầu ra mong muốn cho bài toán tổng thể:\r\nFile output.csv mỗi hàng là kết quả dự đoán mỗi ảnh\r\ncó các cột:\r\nfile_name: tên file 

In [92]:
def format_model_list(df):
    model_strings = []
    for i, row in df.iterrows():
        desc = str(row[1]).replace("{", "{{").replace("}", "}}")
        url = str(row[2])
        model_strings.append(f"{i + 1} - {desc} [More info]({url})")
    return "\n".join(model_strings)


## Tools

In [93]:
def web_scraper(url: str) -> str:
    """Get more details about the model using the model url"""
    # scraper = FirecrawlApp(api_key=FIRECRAWL_API_KEY)
    # scrape_status = scraper.scrape_url(
    #     url,
    #     formats=['markdown']
    # )
    # return scrape_status.markdown
    result = fetching_description_from_huggingface(url)
    return result

In [94]:
class WebScraperParameters(BaseModel):
    url: str

In [95]:
res = web_scraper('https://huggingface.co/thanhtlx/image_classification_01')

In [96]:
tools = [
    Tool(
        name="web_scrapper",
        func=web_scraper,
        description="Use to get more detail about the model",
        args_schema=WebScraperParameters
    )
]

## Model

In [97]:
llm = ChatOpenAI( 
    base_url=BASE_URL,
    model=MODEL,
    api_key=OPENAI_API_KEY
)

In [98]:
class Tasks(BaseModel):
    subtask_one: str = Field(
        ...,
        description="Detailed description of task 1"
    )
    subtask_two: str = Field(
        ...,
        description="Detailed description of task 2"
    )
    subtask_three: str = Field(
        ...,
        description="Detailed description of task 3"
    )

In [99]:
class AgentState(TypedDict):
    task: HumanMessage | None
    data: str | None
    messages: Annotated[BaseMessage, add_messages]
    subtasks: Tasks

In [100]:
def classify_agent(state: AgentState) -> AgentState:
    structured_llm = llm.with_structured_output(Tasks)
    system_prompt = """
                    You are an expert in task formulation. Given a machine learning task, you will
                    help divide the task into these subtasks:
                    - Subtask1: for classifying the problem type and task description
                    - Subtask2: for identifying the input of the problem
                    - Subtask3: for identifying the output of the problem
                    describe the each subtask as specifically as possible
                    """
    messages = [SystemMessage(content=system_prompt)]
    messages.append(state['task'])
    result = structured_llm.invoke(messages)
    state['subtasks'] = result
    return state

In [101]:
state = {
    "task": HumanMessage(content=task),
}

In [102]:
state = classify_agent(state)

# Modeling Agent

In [103]:
class ModelInfo(BaseModel):
    model: str = Field(..., description="Model name")
    url: str = Field(..., description="URL to the model")

class ModelSelection(BaseModel):
    models: List[ModelInfo] = Field(..., description="List of available models with name and URL")

In [104]:
model_selection_parser = PydanticOutputParser(pydantic_object=ModelSelection)

In [105]:
def model_selection_agent(state: AgentState):
    system_prompt = """
                    You are a machine learning expert assigned to select the suitable model for a given task.
                    Given:
                    - A user description about the task
                    - A list of available model (names and links)
                    Your job is to choose ALL suitable models provided in the list for user's specific tasks.
                    Return the answer in the format
                    {structured_output}
                    Here are the list of model:
                    {model_list}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("user", "{input}")
    ]).partial(structured_output=model_selection_parser.get_format_instructions())

    chain = prompt | llm | model_selection_parser

    result = chain.invoke({"input": task, "model_list": format_model_list(df_model)})
    return result

In [106]:
res = model_selection_agent(state)

C:\Users\viett\AppData\Local\Temp\ipykernel_33328\7332665.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  desc = str(row[1]).replace("{", "{{").replace("}", "}}")
C:\Users\viett\AppData\Local\Temp\ipykernel_33328\7332665.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  url = str(row[2])


In [107]:
pprint.pprint(res.models)

[ModelInfo(model='Nhận diện ảnh chữ viết tay', url='https://huggingface.co/thanhtlx/image_classification_01')]


In [108]:
def model_selection_using_tool_agent(state: AgentState):
    system_prompt = """
                    You are a machine learning expert assigned to select the **Best-fit model** for a given task.
                    Given:
                    - A user task description
                    - A list of available models (names and url)
                    - A tool that fetches model details from a provided URL (it help gathers model description, input/output format, code sample usage)

                    Your job is to:
                    1. Use the **provided tool** to retrieve real details about each candidate model:
                        - ✅ Model description
                        - 📥 Input format
                        - 📤 Output format
                        - 🛠️ Library requirements
                        - 🧪 Code sample (usage code snippet)
                    2. Select the best model for user given task based on the data that the tool gives you.

                    ⚠️ VERY IMPORTANT RULES

                    - ❌ DO NOT assume or invent any part of the model's description, input/output format, or code
                    - ❌ DO NOT generate fake code or use your own knowledge about the model
                    - ✅ ONLY use the **actual output** returned from the tool
                    - ✅ Include tool content in your final answer exactly as returned (especially code)

                    ## 🧠 Output Structure (Final Answer)

                    After retrieving tool results, choose ONE best-fit model and output the result in the following format:

                    **✅ Model name and link**  
                    `<model name>` — `<link>`

                    **📝 Description (from tool):**  
                    <model description>

                    **📥 Input format (from tool):**  
                    <description of expected input>

                    **📤 Output format (from tool):**  
                    <description of model output>

                    **🛠️ Library Requirements (from tool) **
                    <requirements to use the model>

                    **🧪 Example code (from tool):**  
                    ```python
                    <exact code snippet from tool>

                    Here are the list of models and there corresponding URL:
                    {model_list}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("ai", "{agent_scratchpad}"),
        ("human", "{input}")
    ])

    agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    task_input = state["task"].content if isinstance(state["task"], SystemMessage) else state["task"]

    # result = executor.invoke({"input": task_input})
    result = executor.invoke({"input": task_input, "model_list": res.models})

    return {"output": result["output"]}

In [109]:
res2 = model_selection_using_tool_agent(state)



> Entering new AgentExecutor chain...

Invoking: `web_scrapper` with `{'url': 'https://huggingface.co/thanhtlx/image_classification_01'}`



Image Classification Model - Handwritten Digit Recognition (Color Images)
📝 Mô tả
Đây là một mô hình phân loại ảnh được huấn luyện để nhận diện các chữ số viết tay (từ 0 đến 9) trong ảnh màu. Mô hình sử dụng kiến trúc Vision Transformer (ViT) và được huấn luyện với tập dữ liệu gồm các chữ số viết tay được chụp hoặc scan dưới dạng ảnh màu.
📌 Chi tiết các mô hình
Phân loại ảnh chữ số viết tay từ 0 đến 9
📥 Đầu vào
Định dạng: Ảnh màu (RGB)
Kiểu dữ liệu: PIL.Image.Image
Kích thước ảnh: Mô hình sẽ tự resize theo cấu hình của ViTImageProcessor (thường là 224x224)
Giá trị pixel: [0, 255] (chuẩn hóa tự động bởi processor)
📤 Đầu ra
Một số nguyên từ 0 đến 9, là kết quả phân loại tương ứng với chữ số xuất hiện trong ảnh.
🛠 Yêu cầu thư viện
Cài đặt các thư viện sau bằng pip:
pip install transformers torch
🧪 Sử dụng mô hình
Dưới đây là đoạn mã mẫu để sử dụng 

In [110]:
pprint.pprint(res2['output'])

('**✅ Model name and link**  \n'
 ' `Nhận diện ảnh chữ viết tay` — '
 '`https://huggingface.co/thanhtlx/image_classification_01`\n'
 '\n'
 '**📝 Description (from tool):**  \n'
 ' Đây là một mô hình phân loại ảnh được huấn luyện để nhận diện các chữ số '
 'viết tay (từ0 đến9) trong ảnh màu. Mô hình sử dụng kiến trúc Vision '
 'Transformer (ViT) và được huấn luyện với tập dữ liệu gồm các chữ số viết tay '
 'được chụp hoặc scan dưới dạng ảnh màu.\n'
 '\n'
 '**📥 Input format (from tool):**  \n'
 ' Định dạng: Ảnh màu (RGB)\n'
 ' Kiểu dữ liệu: PIL.Image.Image\n'
 ' Kích thước ảnh: Mô hình sẽ tự resize theo cấu hình của ViTImageProcessor '
 '(thường là224x224)\n'
 ' Giá trị pixel: [0,255] (chuẩn hóa tự động bởi processor)\n'
 '\n'
 '**📤 Output format (from tool):**  \n'
 ' Một số nguyên từ0 đến9, là kết quả phân loại tương ứng với chữ số xuất hiện '
 'trong ảnh.\n'
 '\n'
 '**🛠️ Library Requirements (from tool) **  \n'
 ' Cài đặt các thư viện sau bằng pip:\n'
 ' pip install transformers torch\

## Output Parser Agent

In [111]:
class ModelingOutput(BaseModel):
    model_description: str = Field(
        ...,
        description="The detailed description of the model."
    )
    model_input_format: str = Field(
        ...,
        description="Model's detailed input format"
    )
    model_output_format: str = Field(
        ...,
        description="Model's detailed output format"
    )
    model_requirements: str = Field(
        ...,
        description="Requirement libraries need to install to use the model."
    )
    model_sample_code: str = Field(
        ...,
        description="Example code to use the model"
    )

In [112]:
parser = PydanticOutputParser(pydantic_object=ModelingOutput)

In [113]:
output_parser_llm = ChatOpenAI(
    base_url=BASE_URL,
    model=MODEL,
    api_key=OPENAI_API_KEY,
    temperature=0
)

In [114]:
def output_parser_agent(state: AgentState):
    system_prompt = """
                    You are a smart AI tasked with extracting structured technical details about a machine learning model from a reasoning result.
                    You are given a detailed text description about a model.
                    Your goal is to fill the following fields based on the text:

                    - `model_description`: A detailed explanation of what the model is and what it does.
                    - `model_input_format`: A detailed description of the model's input format, including dimensions, data types, and expected preprocessing if mentioned.
                    - `model_output_format`: A detailed description of the output format including dimensions, data types, label name and its meaning.
                    - `model_requirements`: A detailed description about the requirements needed to be sastified in order to use the model
                    - `model_sample_code`: A sample code on how to use the model.

                    You MUSTN'T create any data on your own, only using the data provided in the text.

                    Return the data as a JSON object matching the following structure:
                    {formatted_output}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ('system', system_prompt),
        ('human', "{input}")
    ]).partial(formatted_output=parser.get_format_instructions())
    chain = prompt | llm | parser
    response = chain.invoke({"input": res2['output']})
    return response

In [115]:
ans = output_parser_agent(state)

In [116]:
pprint.pprint(res2['output'])

('**✅ Model name and link**  \n'
 ' `Nhận diện ảnh chữ viết tay` — '
 '`https://huggingface.co/thanhtlx/image_classification_01`\n'
 '\n'
 '**📝 Description (from tool):**  \n'
 ' Đây là một mô hình phân loại ảnh được huấn luyện để nhận diện các chữ số '
 'viết tay (từ0 đến9) trong ảnh màu. Mô hình sử dụng kiến trúc Vision '
 'Transformer (ViT) và được huấn luyện với tập dữ liệu gồm các chữ số viết tay '
 'được chụp hoặc scan dưới dạng ảnh màu.\n'
 '\n'
 '**📥 Input format (from tool):**  \n'
 ' Định dạng: Ảnh màu (RGB)\n'
 ' Kiểu dữ liệu: PIL.Image.Image\n'
 ' Kích thước ảnh: Mô hình sẽ tự resize theo cấu hình của ViTImageProcessor '
 '(thường là224x224)\n'
 ' Giá trị pixel: [0,255] (chuẩn hóa tự động bởi processor)\n'
 '\n'
 '**📤 Output format (from tool):**  \n'
 ' Một số nguyên từ0 đến9, là kết quả phân loại tương ứng với chữ số xuất hiện '
 'trong ảnh.\n'
 '\n'
 '**🛠️ Library Requirements (from tool) **  \n'
 ' Cài đặt các thư viện sau bằng pip:\n'
 ' pip install transformers torch\

In [117]:
pprint.pprint(ans.model_description)
pprint.pprint(ans.model_input_format)
pprint.pprint(ans.model_output_format)
pprint.pprint(ans.model_requirements)
pprint.pprint(ans.model_sample_code)

('This is an image classification model trained to recognize handwritten '
 'digits (from 0 to 9) in color images. The model uses the Vision Transformer '
 '(ViT) architecture and was trained on a dataset of handwritten digits '
 'captured or scanned as color images.')
("The model's input format is a color image (RGB) of type PIL.Image.Image. The "
 'image size is automatically resized to 224x224 by the ViTImageProcessor. '
 'Pixel values range from [0, 255] and are automatically normalized by the '
 'processor.')
("The model's output format is an integer from 0 to 9, representing the "
 'classification result corresponding to the digit appearing in the image.')
('The required libraries are transformers and torch, which can be installed '
 'using pip: pip install transformers torch')
('```python\n'
 'from transformers import ViTForImageClassification, ViTImageProcessor\n'
 'from PIL import Image\n'
 'import torch\n'
 'model_name = "thanhtlx/image_classification_01"\n'
 'model = ViTForI

In [118]:
def format_model_list(df):
    model_strings = []
    for i, row in df.iterrows():
        desc = str(row[1]).replace("{", "{{").replace("}", "}}")
        url = str(row[2])
        model_strings.append(f"{i + 1} - {desc} [More info]({url})")
    return "\n".join(model_strings)


In [119]:
class AgentCodingState(TypedDict):
    problem_description: str | None
    problem_input_description: str | None
    problem_output_description: str | None
    model_description: str | None
    model_input_format: str | None
    model_output_format: str | None
    model_requirements: str | None
    model_sample_code: str | None
    output_classes: str | None
    code: str | None


In [ ]:
# %% [markdown]
# # Variables

# %% [code]
OPENAI_API_KEY='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'
FIRECRAWL_API_KEY='fc-e36282d62d1c4ee8971ec0649f3862c7'
BASE_URL='https://api.groq.com/openai/v1'
MODEL="meta-llama/llama-4-scout-17b-16e-instruct"

# %% [markdown]
# # Import Modules

# %% [code]
import sys
sys.path.append('..')
print(sys.path)
from tools.fetching_description_from_huggingface import fetching_description_from_huggingface
from typing import TypedDict, Annotated, List
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.agents import create_tool_calling_agent, AgentExecutor
from pydantic import BaseModel, Field
from langchain.tools import Tool
from firecrawl import FirecrawlApp
from langchain_core.output_parsers import PydanticOutputParser
import pandas as pd
import pprint

# %% [markdown]
# # Testing

# %% [code]
df_task = pd.read_csv("data/ISE - AutoCode Challenge 2 Public - Public task.csv")
df_model = pd.read_csv("data/ISE - AutoCode Challenge 2 Public - Model zoo.csv")

# %% [code]
df_task

# %% [code]
task = df_task["Task"][0]

# %% [code]
task

# %% [code]
def format_model_list(df):
    model_strings = []
    for i, row in df.iterrows():
        desc = str(row[1]).replace("{", "{{").replace("}", "}}")
        url = str(row[2])
        model_strings.append(f"{i + 1} - {desc} [More info]({url})")
    return "\n".join(model_strings)


# %% [markdown]
# ## Tools

# %% [code]
def web_scraper(url: str) -> str:
    """Get more details about the model using the model url"""
    # scraper = FirecrawlApp(api_key=FIRECRAWL_API_KEY)
    # scrape_status = scraper.scrape_url(
    #     url,
    #     formats=['markdown']
    # )
    # return scrape_status.markdown
    result = fetching_description_from_huggingface(url)
    return result

# %% [code]
class WebScraperParameters(BaseModel):
    url: str

# %% [code]
res = web_scraper('https://huggingface.co/thanhtlx/image_classification_01')

# %% [code]
tools = [
    Tool(
        name="web_scrapper",
        func=web_scraper,
        description="Use to get more detail about the model",
        args_schema=WebScraperParameters
    )
]

# %% [markdown]
# ## Model

# %% [code]
llm = ChatOpenAI( 
    base_url=BASE_URL,
    model=MODEL,
    api_key=OPENAI_API_KEY
)

# %% [code]
class Tasks(BaseModel):
    subtask_one: str = Field(
        ...,
        description="Detailed description of task 1"
    )
    subtask_two: str = Field(
        ...,
        description="Detailed description of task 2"
    )
    subtask_three: str = Field(
        ...,
        description="Detailed description of task 3"
    )

# %% [code]
class AgentState(TypedDict):
    task: HumanMessage | None
    data: str | None
    messages: Annotated[BaseMessage, add_messages]
    subtasks: Tasks

# %% [code]
def classify_agent(state: AgentState) -> AgentState:
    structured_llm = llm.with_structured_output(Tasks)
    system_prompt = """
                    You are an expert in task formulation. Given a machine learning task, you will
                    help divide the task into these subtasks:
                    - Subtask1: for classifying the problem type and task description
                    - Subtask2: for identifying the input of the problem
                    - Subtask3: for identifying the output of the problem
                    describe the each subtask as specifically as possible
                    """
    messages = [SystemMessage(content=system_prompt)]
    messages.append(state['task'])
    result = structured_llm.invoke(messages)
    state['subtasks'] = result
    return state

# %% [code]
state = {
    "task": HumanMessage(content=task),
}

# %% [code]
state = classify_agent(state)

# %% [markdown]
# # Modeling Agent

# %% [code]
class ModelInfo(BaseModel):
    model: str = Field(..., description="Model name")
    url: str = Field(..., description="URL to the model")

class ModelSelection(BaseModel):
    models: List[ModelInfo] = Field(..., description="List of available models with name and URL")

# %% [code]
model_selection_parser = PydanticOutputParser(pydantic_object=ModelSelection)

# %% [code]
def model_selection_agent(state: AgentState):
    system_prompt = """
                    You are a machine learning expert assigned to select the suitable model for a given task.
                    Given:
                    - A user description about the task
                    - A list of available model (names and links)
                    Your job is to choose ALL suitable models provided in the list for user's specific tasks.
                    Return the answer in the format
                    {structured_output}
                    Here are the list of model:
                    {model_list}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("user", "{input}")
    ]).partial(structured_output=model_selection_parser.get_format_instructions())

    chain = prompt | llm | model_selection_parser

    result = chain.invoke({"input": task, "model_list": format_model_list(df_model)})
    return result

# %% [code]
res = model_selection_agent(state)

# %% [code]
pprint.pprint(res.models)

# %% [code]
def model_selection_using_tool_agent(state: AgentState):
    system_prompt = """
                    You are a machine learning expert assigned to select the **Best-fit model** for a given task.
                    Given:
                    - A user task description
                    - A list of available models (names and url)
                    - A tool that fetches model details from a provided URL (it help gathers model description, input/output format, code sample usage)

                    Your job is to:
                    1. Use the **provided tool** to retrieve real details about each candidate model:
                        - ✅ Model description
                        - 📥 Input format
                        - 📤 Output format
                        - 🛠️ Library requirements
                        - 🧪 Code sample (usage code snippet)
                    2. Select the best model for user given task based on the data that the tool gives you.

                    ⚠️ VERY IMPORTANT RULES

                    - ❌ DO NOT assume or invent any part of the model's description, input/output format, or code
                    - ❌ DO NOT generate fake code or use your own knowledge about the model
                    - ✅ ONLY use the **actual output** returned from the tool
                    - ✅ Include tool content in your final answer exactly as returned (especially code)

                    ## 🧠 Output Structure (Final Answer)

                    After retrieving tool results, choose ONE best-fit model and output the result in the following format:

                    **✅ Model name and link**  
                    `<model name>` — `<link>`

                    **📝 Description (from tool):**  
                    <model description>

                    **📥 Input format (from tool):**  
                    <description of expected input>

                    **📤 Output format (from tool):**  
                    <description of model output>

                    **🛠️ Library Requirements (from tool) **
                    <requirements to use the model>

                    **🧪 Example code (from tool):**  
                    ```python
                    <exact code snippet from tool>

                    Here are the list of models and there corresponding URL:
                    {model_list}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("ai", "{agent_scratchpad}"),
        ("human", "{input}")
    ])

    agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    task_input = state["task"].content if isinstance(state["task"], SystemMessage) else state["task"]

    # result = executor.invoke({"input": task_input})
    result = executor.invoke({"input": task_input, "model_list": res.models})

    return {"output": result["output"]}

# %% [code]
res2 = model_selection_using_tool_agent(state)

# %% [code]
pprint.pprint(res2['output'])

# %% [markdown]
# ## Output Parser Agent

# %% [code]
class ModelingOutput(BaseModel):
    model_description: str = Field(
        ...,
        description="The detailed description of the model."
    )
    model_input_format: str = Field(
        ...,
        description="Model's detailed input format"
    )
    model_output_format: str = Field(
        ...,
        description="Model's detailed output format"
    )
    model_requirements: str = Field(
        ...,
        description="Requirement libraries need to install to use the model."
    )
    model_sample_code: str = Field(
        ...,
        description="Example code to use the model"
    )

# %% [code]
parser = PydanticOutputParser(pydantic_object=ModelingOutput)

# %% [code]
output_parser_llm = ChatOpenAI(
    base_url=BASE_URL,
    model=MODEL,
    api_key=OPENAI_API_KEY,
    temperature=0
)

# %% [code]
def output_parser_agent(state: AgentState):
    system_prompt = """
                    You are a smart AI tasked with extracting structured technical details about a machine learning model from a reasoning result.
                    You are given a detailed text description about a model.
                    Your goal is to fill the following fields based on the text:

                    - `model_description`: A detailed explanation of what the model is and what it does.
                    - `model_input_format`: A detailed description of the model's input format, including dimensions, data types, and expected preprocessing if mentioned.
                    - `model_output_format`: A detailed description of the output format including dimensions, data types, label name and its meaning.
                    - `model_requirements`: A detailed description about the requirements needed to be sastified in order to use the model
                    - `model_sample_code`: A sample code on how to use the model.

                    You MUSTN'T create any data on your own, only using the data provided in the text.

                    Return the data as a JSON object matching the following structure:
                    {formatted_output}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ('system', system_prompt),
        ('human', "{input}")
    ]).partial(formatted_output=parser.get_format_instructions())
    chain = prompt | llm | parser
    response = chain.invoke({"input": res2['output']})
    return response

# %% [code]
ans = output_parser_agent(state)

# %% [code]
pprint.pprint(res2['output'])

# %% [code]
pprint.pprint(ans.model_description)
pprint.pprint(ans.model_input_format)
pprint.pprint(ans.model_output_format)
pprint.pprint(ans.model_requirements)
pprint.pprint(ans.model_sample_code)

# %% [code]


# %% [code]
def format_model_list(df):
    model_strings = []
    for i, row in df.iterrows():
        desc = str(row[1]).replace("{", "{{").replace("}", "}}")
        url = str(row[2])
        model_strings.append(f"{i + 1} - {desc} [More info]({url})")
    return "\n".join(model_strings)


# %% [code]
class AgentCodingState(TypedDict):
    problem_description: str | None
    problem_input_description: str | None
    problem_output_description: str | None
    model_description: str | None
    model_input_format: str | None
    model_output_format: str | None
    model_requirements: str | None
    model_sample_code: str | None
    output_classes: str | None
    code: str | None


# %% [code]
base_prompt = r"""
    You are a specialist in Machine Learning. Your task is to generate a *fully functional with all necessary imports and dependencies* codebase in *Python* that can be executed flawlessly.

    You will be provided with:
    - A problem description
    - An input specification
    - An output specification
    - A model description

    ### Input:
    - Problem description: {problem_description}
    - Problem input specification: {problem_input_description}
    - Problem output specification: {problem_output_description}
    - ML model description: {model_description}
    - ML model input format: {model_input_format}
    - ML model output format: {model_output_format}
    - ML model requirements: {model_requirements}
    - ML model sample code: {model_sample_code}
    - Output classes: {output_classes}

    ### Guidelines

    You *must* strictly follow the following guidelines:
    - The preprocessing step should be suitable for the data type.
    - The postprocessing step should notices the differences between the data returned by the model and the output requirements. You must extract and use the exact class labels as defined in the output specification.
    - Do *not invent new labels or translate* the class names. Use them exactly as given.
    - You *must* make sure that your codebase can be executed flawlessly that would not encounter any errors or exceptions.

    Your implementation *must strictly follow* the structure below:
    1. *Imports*: All required libraries.
    2. *Preprocessing*: Handle and transform the input as defined.
    3. *Inference logic*: Use the described model for prediction. You *must* use tqdm or similar logging library to track progress.
    4. *Postprocessing*: Format or transform the raw output into the final result as described.

    You must *not* include any explanations, markdown, or logging outside what is required by the problem.

    Return *only* the complete Python codebase. Wrap it with:
    \`\`\`python
    # code here
    \`\`\`
    """


# %% [code]
class OutputState(TypedDict):
    output_classes: str | None

# %% [code]
def output_agent(state: OutputState) -> OutputState:
    prompt = r"""
        Your are a specialist in machine learning. Your task is to identify the absolute classes of the given problem description, following with an output description.
        ### Input:
        - Problem description: {problem_description}
        - Output description: {output_description}

        You must return an array, strictly following these guidelines:
        - Understand the context from the given problem description.
        - Extract the class names from the output description. You **must not** invent new labels or translate the class names. Use them exactly as given in the output description.
        - Create an array containing the classes.

        You must return only the array containing those classes, without any formatting.
    """

    prompt = prompt.format(
        problem_description = state["problem_description"],
        output_description = state["output_description"]
    )
    response = llm.invoke(prompt)
    return {**state, "output_classes": response.content}

# %% [code]
output_state = {
    "problem_description": task,
    "output_description": ans.model_output_format
}

# %% [code]
new_state = output_agent(output_state)

# %% [code]
new_state["output_classes"]

# %% [code]
def coding_agent(state: AgentCodingState) -> AgentCodingState:
    prompt = base_prompt.format(
        problem_description=state["problem_description"],
        problem_input_description=state["problem_input_description"],
        problem_output_description=state["problem_output_description"],
        model_description=state["model_description"],
        model_input_format=state["model_input_format"],
        model_output_format=state["model_output_format"],
        model_requirements=state["model_requirements"],
        model_sample_code=state["model_sample_code"],
        output_classes=state["output_classes"],
    )
    response = llm.invoke(prompt)
    return {**state, "code": response.content}

# %% [code]


# %% [code]
prev_state = {
    "problem_description": task,
    "problem_input_description": ans.problem_input_description,
    "problem_output_description": ans.problem_output_description,
    "model_description": res2['output'],
    "model_input_format": ans.model_input_format,
    "model_output_format": ans.model_output_format,
    "model_requirements": ans.model_requirements,
    "model_sample_code": ans.model_sample_code,
    "output_classes": ans.output_classes,
}


# %% [code]
state = coding_agent(prev_state)

In [121]:
class OutputState(TypedDict):
    output_classes: str | None

In [122]:
def output_agent(state: OutputState) -> OutputState:
    prompt = r"""
        Your are a specialist in machine learning. Your task is to identify the absolute classes of the given problem description, following with an output description.
        ### Input:
        - Problem description: {problem_description}
        - Output description: {output_description}

        You must return an array, strictly following these guidelines:
        - Understand the context from the given problem description.
        - Extract the class names from the output description. You **must not** invent new labels or translate the class names. Use them exactly as given in the output description.
        - Create an array containing the classes.

        You must return only the array containing those classes, without any formatting.
    """

    prompt = prompt.format(
        problem_description = state["problem_description"],
        output_description = state["output_description"]
    )
    response = llm.invoke(prompt)
    return {**state, "output_classes": response.content}

In [123]:
output_state = {
    "problem_description": task,
    "output_description": ans.model_output_format
}

In [124]:
new_state = output_agent(output_state)

In [125]:
new_state["output_classes"]

'[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]'

In [126]:
def coding_agent(state: AgentCodingState) -> AgentCodingState:
    prompt = base_prompt.format(
        problem_description=state["problem_description"],
        problem_input_description=state["problem_input_description"],
        problem_output_description=state["problem_output_description"],
        model_description=state["model_description"],
        model_input_format=state["model_input_format"],
        model_output_format=state["model_output_format"],
        model_requirements=state["model_requirements"],
        model_sample_code=state["model_sample_code"],
        output_classes=state["output_classes"],
    )
    response = llm.invoke(prompt)
    return {**state, "code": response.content}

In [127]:
prev_state = {
    "problem_description": task,
    "problem_input_description": ans.problem_input_description,
    "problem_output_description": ans.problem_output_description,
    "model_description": res2['output'],
    "model_input_format": ans.model_input_format,
    "model_output_format": ans.model_output_format,
    "model_requirements": ans.model_requirements,
    "model_sample_code": ans.model_sample_code,
    "output_classes": ans.output_classes,
}


AttributeError: 'ModelingOutput' object has no attribute 'problem_input_description'

In [ ]:
state = coding_agent(prev_state)